<div style="text-align: center;">
   <h1>TD&TP 4 : Développement d'une Application de Gestion de Mots de Passe Sécurisée avec Journalisation </h1> 
</div>

## Contexte
Dans le cadre de ce projet, vous allez développer une application Python sécurisée de gestion de
mots de passe. L'objectif est de mettre en pratique les concepts de sécurité appris précédemment,
en intégrant des techniques de validation des entrées, de gestion sécurisée des mots de passe, de
prévention des injections SQL, et de chiffrement des données sensibles. Vous allez également
implémenter un système de journalisation modulaire et flexible en utilisant des décorateurs.

## Fichier `config.py`
Ce fichier gère la configuration de l'application, notamment :
- La configuration de la base de données (`DATABASE_PATH`).
- La génération et la gestion d'une clé de chiffrement pour protéger les mots de passe (`get_encryption_key`).
- La configuration du système de journalisation (`LOGGING_CONFIG_PATH`).

In [2]:
# config.py
import os
from cryptography.fernet import Fernet

class Config:
    # Configuration de la base de données
    DATABASE_PATH = "passwords.db"
    
    # Génération et stockage de la clé de chiffrement
    @staticmethod
    def get_encryption_key():
        key_file = "encryption_key.key"
        if os.path.exists(key_file):
            with open(key_file, "rb") as f:
                return f.read()
        else:
            key = Fernet.generate_key()
            with open(key_file, "wb") as f:
                f.write(key)
            return key
    
    # Configuration du logging
    LOGGING_CONFIG_PATH = "logging_config.yaml"

## Fichier `database.py`
Il s'occupe de la gestion de la base de données SQLite, incluant :
- La connexion à la base de données (`get_db_connection`).
- L'initialisation des tables nécessaires (utilisateurs et mots de passe stockés) avec la fonction `initialize_database`.
- Les fonctions permettant de manipuler les enregistrements des utilisateurs et des mots de passe stockés.

In [ ]:
# database.py
import sqlite3
import logging
from config import Config

logger = logging.getLogger(__name__)

def get_db_connection():
    """Établir une connexion à la base de données SQLite."""
    try:
        conn = sqlite3.connect(Config.DATABASE_PATH)
        conn.row_factory = sqlite3.Row
        return conn
    except sqlite3.Error as e:
        logger.error(f"Erreur lors de la connexion à la base de données: {e}")
        raise

def initialize_database():
    """Créer les tables nécessaires si elles n'existent pas."""
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        # Table des utilisateurs
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL,
            password_hash TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Table des mots de passe stockés
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS passwords (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER NOT NULL,
            site_name TEXT NOT NULL,
            username TEXT NOT NULL,
            encrypted_password TEXT NOT NULL,
            notes TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES users (id)
        )
        ''')
        
        conn.commit()
        logger.info("Base de données initialisée avec succès")
    except sqlite3.Error as e:
        logger.error(f"Erreur lors de l'initialisation de la base de données: {e}")
        raise
    finally:
        conn.close()

def add_user(username, password_hash, email):
    """Ajouter un nouvel utilisateur à la base de données."""
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        cursor.execute(
            "INSERT INTO users (username, password_hash, email) VALUES (?, ?, ?)",
            (username, password_hash, email)
        )
        
        conn.commit()
        user_id = cursor.lastrowid
        logger.info(f"Nouvel utilisateur créé avec ID: {user_id}")
        return user_id
    except sqlite3.IntegrityError:
        logger.error(f"L'utilisateur {username} ou l'email {email} existe déjà")
        raise ValueError(f"L'utilisateur {username} ou l'email {email} existe déjà")
    except sqlite3.Error as e:
        logger.error(f"Erreur lors de l'ajout de l'utilisateur: {e}")
        raise
    finally:
        conn.close()

def get_user_by_username(username):
    """Récupérer les informations d'un utilisateur par son nom d'utilisateur."""
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        cursor.execute("SELECT * FROM users WHERE username = ?", (username,))
        user = cursor.fetchone()
        
        return dict(user) if user else None
    except sqlite3.Error as e:
        logger.error(f"Erreur lors de la récupération de l'utilisateur: {e}")
        raise
    finally:
        conn.close()

def add_password(user_id, site_name, username, encrypted_password, notes=None):
    """Ajouter un nouveau mot de passe pour un utilisateur."""
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        cursor.execute(
            "INSERT INTO passwords (user_id, site_name, username, encrypted_password, notes) VALUES (?, ?, ?, ?, ?)",
            (user_id, site_name, username, encrypted_password, notes)
        )
        
        conn.commit()
        password_id = cursor.lastrowid
        logger.info(f"Nouveau mot de passe ajouté avec ID: {password_id}")
        return password_id
    except sqlite3.Error as e:
        logger.error(f"Erreur lors de l'ajout du mot de passe: {e}")
        raise
    finally:
        conn.close()

def get_passwords_by_user_id(user_id):
    """Récupérer tous les mots de passe d'un utilisateur."""
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        cursor.execute("SELECT * FROM passwords WHERE user_id = ?", (user_id,))
        passwords = cursor.fetchall()
        
        return [dict(pw) for pw in passwords]
    except sqlite3.Error as e:
        logger.error(f"Erreur lors de la récupération des mots de passe: {e}")
        raise
    finally:
        conn.close()

def get_password_by_id(password_id, user_id):
    """Récupérer un mot de passe spécifique d'un utilisateur."""
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        cursor.execute("SELECT * FROM passwords WHERE id = ? AND user_id = ?", (password_id, user_id))
        password = cursor.fetchone()
        
        return dict(password) if password else None
    except sqlite3.Error as e:
        logger.error(f"Erreur lors de la récupération du mot de passe: {e}")
        raise
    finally:
        conn.close()

# Fichier `decorators.py`
Ce fichier contient des décorateurs pour enrichir certaines fonctions de l'application :
- `log_function_call` : Un décorateur qui journalise les appels de fonctions, enregistre les arguments passés, et mesure le temps d'exécution.
  - Il masque les informations sensibles comme les mots de passe dans les journaux.
- `requires_auth` : Un décorateur qui vérifie l'authentification des utilisateurs avant de leur permettre d'accéder à des fonctions protégées. Si l'utilisateur n'est pas authentifié, il lève une erreur.


In [1]:
# decorators.py
import logging
import time
from typing import Callable, Any

logger = logging.getLogger(__name__)

def log_function_call(func: Callable) -> Callable:
    """
    Décorateur pour journaliser les appels de fonction.
    Enregistre les informations sur l'entrée et la sortie de la fonction,
    ainsi que le temps d'exécution.
    """

    def wrapper(*args, **kwargs):
        func_name = func.__name__
        logger.info(f"Appel de la fonction {func_name}")
        
        try:
            # Masquer les informations sensibles dans les logs
            safe_args = ["*****" if isinstance(arg, str) and len(arg) > 10 else arg for arg in args]
            safe_kwargs = {k: "*****" if isinstance(v, str) and len(v) > 10 else v for k, v in kwargs.items()}
            
            logger.debug(f"Arguments: {safe_args}, Kwargs: {safe_kwargs}")
            
            start_time = time.time()
            result = func(*args, **kwargs)
            execution_time = time.time() - start_time
            
            logger.info(f"Fonction {func_name} exécutée en {execution_time:.4f} secondes")
            return result
        except Exception as e:
            logger.error(f"Erreur lors de l'exécution de {func_name}: {str(e)}")
            raise
    
    return wrapper

def requires_auth(func: Callable) -> Callable:
    """
    Décorateur pour vérifier l'authentification de l'utilisateur.
    Vérifie si l'utilisateur est connecté avant d'autoriser l'accès à la fonction.
    """
    def wrapper(self, *args, **kwargs):
        if not hasattr(self, 'current_user') or self.current_user is None:
            logger.warning("Tentative d'accès non autorisé à une fonction protégée")
            raise PermissionError("Authentification requise pour accéder à cette fonctionnalité")
        
        logger.info(f"Accès autorisé pour l'utilisateur {self.current_user['username']} à la fonction {func.__name__}")
        return func(self, *args, **kwargs)
    
    return wrapper

## Fichier `logging_config.yaml`
Fichier de configuration YAML pour le système de journalisation, définissant :
- Les différents gestionnaires de journaux (console, fichiers de logs).
- Les niveaux de journalisation (DEBUG, INFO, ERROR) et les formats associés.

In [4]:
# logging_config.py
import os
import yaml
import logging
import logging.config

def setup_logging(config_path="logging_config.yaml", default_level=logging.INFO):
    """
    Configure la journalisation à partir d'un fichier YAML.
    """
    if os.path.exists(config_path):
        with open(config_path, 'rt') as f:
            try:
                config = yaml.safe_load(f.read())
                logging.config.dictConfig(config)
            except Exception as e:
                print(f"Erreur lors du chargement de la configuration de journalisation: {e}")
                logging.basicConfig(level=default_level)
    else:
        logging.basicConfig(level=default_level)
        print(f"Fichier de configuration {config_path} non trouvé. Utilisation de la configuration par défaut.")

# Créer le fichier de configuration de journalisation par défaut s'il n'existe pas
def create_default_logging_config():
    """
    Crée un fichier de configuration YAML par défaut pour la journalisation.
    """
    config = {
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'standard': {
                'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            },
            'detailed': {
                'format': '%(asctime)s - %(name)s - %(levelname)s - %(filename)s:%(lineno)d - %(message)s'
            }
        },
        'handlers': {
            'console': {
                'class': 'logging.StreamHandler',
                'level': 'INFO',
                'formatter': 'standard',
                'stream': 'ext://sys.stdout'
            },
            'file': {
                'class': 'logging.handlers.RotatingFileHandler',
                'level': 'DEBUG',
                'formatter': 'detailed',
                'filename': 'password_manager.log',
                'maxBytes': 10485760,  # 10MB
                'backupCount': 5
            },
            'error_file': {
                'class': 'logging.handlers.RotatingFileHandler',
                'level': 'ERROR',
                'formatter': 'detailed',
                'filename': 'error.log',
                'maxBytes': 10485760,  # 10MB
                'backupCount': 5
            }
        },
        'loggers': {
            '': {  # root logger
                'handlers': ['console', 'file', 'error_file'],
                'level': 'DEBUG',
                'propagate': True
            }
        }
    }
    
    with open('logging_config.yaml', 'w') as f:
        yaml.dump(config, f, default_flow_style=False)

## Fichier `models.py`
Ce fichier contient les modèles de données utilisés pour valider les entrées utilisateur via la bibliothèque Pydantic, incluant :
- `UserRegistration` : Valide les données d'enregistrement des utilisateurs, notamment la complexité du mot de passe.
- `UserLogin` : Valide les données de connexion des utilisateurs.
- `PasswordEntry` : Valide les informations de mots de passe (site, nom d'utilisateur, mot de passe).

In [5]:
# models.py
from pydantic import BaseModel, EmailStr, Field, field_validator
import re

class UserRegistration(BaseModel):
    """Modèle pour la validation des données d'enregistrement d'un utilisateur."""
    username: str = Field(..., min_length=3, max_length=50)
    email: EmailStr
    password: str = Field(..., min_length=8)
    
    @field_validator('password')
    def password_strength(cls, v):
        """Valider la complexité du mot de passe."""
        if not re.search(r'[A-Z]', v):
            raise ValueError('Le mot de passe doit contenir au moins une lettre majuscule')
        if not re.search(r'[a-z]', v):
            raise ValueError('Le mot de passe doit contenir au moins une lettre minuscule')
        if not re.search(r'[0-9]', v):
            raise ValueError('Le mot de passe doit contenir au moins un chiffre')
        if not re.search(r'[!@#$%^&*(),.?":{}|<>]', v):
            raise ValueError('Le mot de passe doit contenir au moins un caractère spécial')
        return v

class UserLogin(BaseModel):
    """Modèle pour la validation des données de connexion d'un utilisateur."""
    username: str
    password: str

class PasswordEntry(BaseModel):
    """Modèle pour la validation des entrées de mots de passe à stocker."""
    site_name: str = Field(..., min_length=1, max_length=100)
    username: str = Field(..., min_length=1, max_length=100)
    password: str = Field(..., min_length=1)
    notes: str = Field(None, max_length=500)

## Fichier `security.py`
Ce fichier s'occupe de la sécurité, en particulier du hachage et du chiffrement des mots de passe :
- `hash_password` : Hache un mot de passe en utilisant l'algorithme `bcrypt`.
- `verify_password` : Vérifie si un mot de passe en clair correspond à son hash.
- `encrypt_password` : Chiffre un mot de passe à l'aide de la bibliothèque `Fernet` pour assurer sa confidentialité.
- `decrypt_password` : Déchiffre un mot de passe qui a été chiffré avec `Fernet`.

In [ ]:
# security.py
import bcrypt
import logging
from cryptography.fernet import Fernet
from config import Config

logger = logging.getLogger(__name__)

# Initialiser la clé de chiffrement Fernet
ENCRYPTION_KEY = Config.get_encryption_key()
cipher_suite = Fernet(ENCRYPTION_KEY)

def hash_password(password):
    """Hacher un mot de passe avec bcrypt."""
    try:
        password_bytes = password.encode('utf-8')
        salt = bcrypt.gensalt()
        hashed = bcrypt.hashpw(password_bytes, salt)
        return hashed.decode('utf-8')
    except Exception as e:
        logger.error(f"Erreur lors du hachage du mot de passe: {e}")
        raise

def verify_password(plain_password, hashed_password):
    """Vérifier si un mot de passe correspond à son hash."""
    try:
        plain_password_bytes = plain_password.encode('utf-8')
        hashed_password_bytes = hashed_password.encode('utf-8')
        return bcrypt.checkpw(plain_password_bytes, hashed_password_bytes)
    except Exception as e:
        logger.error(f"Erreur lors de la vérification du mot de passe: {e}")
        return False

def encrypt_password(password):
    """Chiffrer un mot de passe avec Fernet."""
    try:
        password_bytes = password.encode('utf-8')
        encrypted = cipher_suite.encrypt(password_bytes)
        return encrypted.decode('utf-8')
    except Exception as e:
        logger.error(f"Erreur lors du chiffrement du mot de passe: {e}")
        raise

def decrypt_password(encrypted_password):
    """Déchiffrer un mot de passe chiffré avec Fernet."""
    try:
        encrypted_bytes = encrypted_password.encode('utf-8')
        decrypted = cipher_suite.decrypt(encrypted_bytes)
        return decrypted.decode('utf-8')
    except Exception as e:
        logger.error(f"Erreur lors du déchiffrement du mot de passe: {e}")
        raise

## Fichier `main.py`
Il contient la logique principale de l'application, orchestrant les fonctionnalités de gestion des mots de passe :
- Création d'un utilisateur et hachage de son mot de passe.
- Ajout, modification et suppression des mots de passe stockés.
- Chiffrement et déchiffrement des mots de passe lors de leur utilisation.

In [11]:
# main.py
import logging
from typing import Optional, Dict, List, Any
import getpass
import sys

from models import UserRegistration, UserLogin, PasswordEntry
from database import (
    initialize_database, add_user, get_user_by_username,
    add_password, get_passwords_by_user_id, get_password_by_id
)
from security import hash_password, verify_password, encrypt_password, decrypt_password
from decorators import log_function_call, requires_auth
from logging_config import setup_logging, create_default_logging_config

# Initialiser la journalisation
create_default_logging_config()
setup_logging()
logger = logging.getLogger(__name__)

class PasswordManager:
    def __init__(self):
        self.current_user = None
        # Initialiser la base de données
        try:
            initialize_database()
            logger.info("Application de gestion de mots de passe initialisée")
        except Exception as e:
            logger.critical(f"Échec de l'initialisation de l'application: {e}")
            sys.exit(1)
    
    @log_function_call
    def register_user(self, username: str, email: str, password: str) -> bool:
        """Enregistrer un nouvel utilisateur."""
        try:
            # Valider les données utilisateur avec Pydantic
            user_data = UserRegistration(username=username, email=email, password=password)
            
            # Hacher le mot de passe
            hashed_password = hash_password(user_data.password)
            
            # Ajouter l'utilisateur à la base de données
            user_id = add_user(user_data.username, hashed_password, user_data.email)
            
            logger.info(f"Utilisateur {username} enregistré avec succès")
            return True
        except ValueError as e:
            logger.error(f"Erreur de validation: {e}")
            print(f"Erreur: {e}")
            return False
        except Exception as e:
            logger.error(f"Erreur lors de l'enregistrement de l'utilisateur: {e}")
            print(f"Erreur: Une erreur inattendue s'est produite")
            return False
    
    @log_function_call
    def login_user(self, username: str, password: str) -> bool:
        """Connecter un utilisateur existant."""
        try:
            # Valider les données de connexion
            login_data = UserLogin(username=username, password=password)
            
            # Récupérer l'utilisateur depuis la base de données
            user = get_user_by_username(login_data.username)
            if not user:
                logger.warning(f"Tentative de connexion avec un nom d'utilisateur inexistant: {username}")
                print("Erreur: Nom d'utilisateur ou mot de passe incorrect")
                return False
            
            # Vérifier le mot de passe
            if not verify_password(login_data.password, user['password_hash']):
                logger.warning(f"Tentative de connexion avec un mot de passe incorrect pour {username}")
                print("Erreur: Nom d'utilisateur ou mot de passe incorrect")
                return False
            
            # Stocker l'utilisateur connecté
            self.current_user = user
            logger.info(f"Utilisateur {username} connecté avec succès")
            return True
        except Exception as e:
            logger.error(f"Erreur lors de la connexion: {e}")
            print(f"Erreur: Une erreur inattendue s'est produite")
            return False
    
    @requires_auth
    @log_function_call
    def store_password(self, site_name: str, username: str, password: str, notes: Optional[str] = None) -> bool:
        """Stocker un nouveau mot de passe pour l'utilisateur connecté."""
        try:
            # Valider les données du mot de passe
            password_data = PasswordEntry(site_name=site_name, username=username, password=password, notes=notes)
            
            # Chiffrer le mot de passe
            encrypted_password = encrypt_password(password_data.password)
            
            # Ajouter le mot de passe à la base de données
            add_password(
                self.current_user['id'],
                password_data.site_name,
                password_data.username,
                encrypted_password,
                password_data.notes
            )
            
            logger.info(f"Mot de passe pour {site_name} stocké avec succès")
            return True
        except Exception as e:
            logger.error(f"Erreur lors du stockage du mot de passe: {e}")
            print(f"Erreur: Une erreur inattendue s'est produite")
            return False
    
    @requires_auth
    @log_function_call
    def retrieve_passwords(self) -> List[Dict[str, Any]]:
        """Récupérer tous les mots de passe de l'utilisateur connecté."""
        try:
            passwords = get_passwords_by_user_id(self.current_user['id'])
            logger.info(f"Récupération de {len(passwords)} mots de passe pour {self.current_user['username']}")
            return passwords
        except Exception as e:
            logger.error(f"Erreur lors de la récupération des mots de passe: {e}")
            print(f"Erreur: Une erreur inattendue s'est produite")
            return []
    
    @requires_auth
    @log_function_call
    def retrieve_password(self, password_id: int) -> Optional[Dict[str, Any]]:
        """Récupérer et déchiffrer un mot de passe spécifique."""
        try:
            # Récupérer le mot de passe chiffré
            password_entry = get_password_by_id(password_id, self.current_user['id'])
            if not password_entry:
                logger.warning(f"Tentative d'accès à un mot de passe inexistant: ID {password_id}")
                print("Erreur: Mot de passe non trouvé")
                return None
            
            # Déchiffrer le mot de passe
            decrypted_password = decrypt_password(password_entry['encrypted_password'])
            
            # Remplacer le mot de passe chiffré par le mot de passe déchiffré
            password_entry['password'] = decrypted_password
            del password_entry['encrypted_password']
            
            logger.info(f"Mot de passe récupéré avec succès: ID {password_id}")
            return password_entry
        except Exception as e:
            logger.error(f"Erreur lors de la récupération du mot de passe: {e}")
            print(f"Erreur: Une erreur inattendue s'est produite")
            return None
    
    def logout(self) -> None:
        """Déconnecter l'utilisateur actuel."""
        if self.current_user:
            logger.info(f"Déconnexion de l'utilisateur {self.current_user['username']}")
            self.current_user = None
        else:
            logger.warning("Tentative de déconnexion sans utilisateur connecté")

def main():
    """Fonction principale pour l'interface en ligne de commande."""
    password_manager = PasswordManager()
    
    while True:
        if password_manager.current_user is None:
            print("\n--- Gestionnaire de Mots de Passe ---")
            print("1. Connexion")
            print("2. Inscription")
            print("3. Quitter")
            choice = input("Choix: ")
            
            if choice == "1":
                username = input("Nom d'utilisateur: ")
                password = getpass.getpass("Mot de passe: ")
                if password_manager.login_user(username, password):
                    print(f"Bienvenue, {username}!")
            
            elif choice == "2":
                username = input("Nom d'utilisateur: ")
                email = input("Email: ")
                password = getpass.getpass("Mot de passe: ")
                confirm_password = getpass.getpass("Confirmer le mot de passe: ")
                
                if password != confirm_password:
                    print("Les mots de passe ne correspondent pas.")
                    continue
                
                if password_manager.register_user(username, email, password):
                    print("Inscription réussie! Vous pouvez maintenant vous connecter.")
            
            elif choice == "3":
                print("Au revoir!")
                break
            
            else:
                print("Choix invalide, veuillez réessayer.")
        
        else:
            print(f"\n--- Bienvenue, {password_manager.current_user['username']} ---")
            print("1. Stocker un nouveau mot de passe")
            print("2. Afficher tous les mots de passe")
            print("3. Récupérer un mot de passe spécifique")
            
            print("4. Déconnexion")
            choice = input("Choix: ")
            
            if choice == "1":
                site_name = input("Nom du site: ")
                username = input("Nom d'utilisateur: ")
                password = getpass.getpass("Mot de passe: ")
                notes = input("Notes (optionnel): ")
                
                if password_manager.store_password(site_name, username, password, notes):
                    print("Mot de passe stocké avec succès!")
            
            elif choice == "2":
                passwords = password_manager.retrieve_passwords()
                if passwords:
                    print("\n--- Vos mots de passe ---")
                    for pw in passwords:
                        print(f"ID: {pw['id']} | Site: {pw['site_name']} | Utilisateur: {pw['username']}")
                else:
                    print("Aucun mot de passe trouvé.")
            
            elif choice == "3":
                try:
                    password_id = int(input("ID du mot de passe: "))
                    password_entry = password_manager.retrieve_password(password_id)
                    
                    if password_entry:
                        print("\n--- Détails du mot de passe ---")
                        print(f"Site: {password_entry['site_name']}")
                        print(f"Utilisateur: {password_entry['username']}")
                        print(f"Mot de passe: {password_entry['password']}")
                        if password_entry['notes']:
                            print(f"Notes: {password_entry['notes']}")
                except ValueError:
                    print("Veuillez entrer un ID valide.")
            
            elif choice == "4":
                password_manager.logout()
                print("Vous êtes déconnecté.")
            
            else:
                print("Choix invalide, veuillez réessayer.")

if __name__ == "__main__":
    main()

2025-03-19 01:30:17,415 - database - INFO - Base de données initialisée avec succès


2025-03-19 01:30:17,417 - __main__ - INFO - Application de gestion de mots de passe initialisée

--- Gestionnaire de Mots de Passe ---
1. Connexion
2. Inscription
3. Quitter
2025-03-19 01:30:41,074 - decorators - INFO - Appel de la fonction register_user
2025-03-19 01:30:41,339 - database - INFO - Nouvel utilisateur créé avec ID: 1
2025-03-19 01:30:41,341 - __main__ - INFO - Utilisateur belannab mohamed   enregistré avec succès
2025-03-19 01:30:41,342 - decorators - INFO - Fonction register_user exécutée en 0.2663 secondes
Inscription réussie! Vous pouvez maintenant vous connecter.

--- Gestionnaire de Mots de Passe ---
1. Connexion
2. Inscription
3. Quitter
2025-03-19 01:30:51,127 - decorators - INFO - Appel de la fonction login_user
2025-03-19 01:30:51,384 - __main__ - INFO - Utilisateur belannab mohamed   connecté avec succès
2025-03-19 01:30:51,385 - decorators - INFO - Fonction login_user exécutée en 0.2565 secondes
Bienvenue, belannab mohamed  !

--- Bienvenue, belannab mohamed  